# Readability Assessment throughConvolutional Neural Network

Readability assessment is a well known problem in natural language processing field. Giving someone the suitable text for his level of comprehension (not so easy and not so hard) could maximize his understanding and enjoyment. In this notebook we are trying to assess the readability of a given text regardless of the text subject using recurrent neural network.

## Corpus
> OneStopEnglish corpus: A new corpus for automatic readability assessment and text simplification  
> Sowmya Vajjala and Ivana Lučić  
> 2018  
> Proceedings of the Thirteenth Workshop on Innovative Use of NLP for Building Educational Applications, pages 297–304. Association for Computational Linguistics.  
> [url](http://aclweb.org/anthology/W18-0535). [bib file](https://aclanthology.coli.uni-saarland.de/papers/W18-0535/w18-0535.bib)

Please cite the above paper if you use this corpus in your research.

[![DOI](https://zenodo.org/badge/128919409.svg)](https://zenodo.org/badge/latestdoi/128919409)

<a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-sa/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-sa/4.0/">Creative Commons Attribution-ShareAlike 4.0 International License</a>.

Now let's dive into our corpus.

In [2]:
import sys
sys.path.append("/home/ms10596/PycharmProjects/match")
from ipywidgets import interact
from tabulate import tabulate
from IPython.display import display, HTML

from utils.loading import load_glove_embeddings
from utils.one_stop_english import load_corpus, corpus_to_words, corpus_to_pos, detokenize

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense, LSTM, Bidirectional,Conv1D,MaxPooling1D,GlobalMaxPooling1D, Dropout
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.regularizers import l1, l2, l1_l2
from tensorflow.keras.callbacks import EarlyStopping

import numpy as np

corpus = load_corpus()
articles_words, tags = corpus_to_words(corpus)
articles_pos, tags = corpus_to_pos(corpus)

Reading level|Avg. Num. Words|Std. Dev|Number of Articles
---|---|---|---
Elementary|533.17|103.79|189
Intermediate|676.59|117.15|189
Advanced|820.49|162.52|189



In [3]:
@interact
def show_articles(i=(0,188,1), words=(0,1000,1)):
    data = [
        ["Advanced",detokenize(articles_words[i][:words])], 
        ["Intermediate",detokenize(articles_words[i+2][:words])], 
        ["Elementary",detokenize(articles_words[i+1][:words])]
    ]
    headers = ['Reading Level', 'Example']
    display(HTML(tabulate(data,tablefmt='html', headers=headers)+"<style>th,td {font-size: 10px}</style>"))

interactive(children=(IntSlider(value=94, description='i', max=188), IntSlider(value=500, description='words',…

In [4]:
print(articles_words[0][:10])
print(articles_pos[0][:10])

['When', 'you', 'see', 'the', 'word', 'Amazon', ',', 'whats', 'the', 'first']
['WRB', 'PRP', 'VB', 'DT', 'NN', 'NN', ',', 'VBZ', 'DT', 'JJ']


In [5]:
maxlen = 1000 # Cuts off reviews after 1000 words
max_words = 45
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(articles_pos)
print(tokenizer.word_index)

{'nn': 1, 'in': 2, 'dt': 3, 'jj': 4, 'nns': 5, 'nnp': 6, ',': 7, '.': 8, 'rb': 9, 'prp': 10, 'vb': 11, 'vbd': 12, 'cc': 13, 'vbz': 14, 'to': 15, 'vbp': 16, 'cd': 17, 'vbn': 18, 'vbg': 19, 'prp$': 20, 'md': 21, 'wdt': 22, 'wrb': 23, 'wp': 24, 'jjr': 25, 'rp': 26, ':': 27, 'jjs': 28, 'ex': 29, 'rbr': 30, 'nnps': 31, '-rrb-': 32, '-lrb-': 33, 'rbs': 34, 'pdt': 35, '$': 36, 'fw': 37, 'uh': 38, 'wp$': 39, 'sym': 40, "''": 41, 'ls': 42, 'pos': 43, '``': 44}


In [6]:
sequences = tokenizer.texts_to_sequences(articles_pos)
print(np.shape(sequences))
print(sequences[0])

(567,)
[23, 10, 11, 3, 1, 1, 7, 14, 3, 4, 1, 22, 14, 15, 11, 3, 4, 28, 1, 7, 3, 28, 1, 13, 3, 28, 1, 1, 13, 22, 16, 10, 16, 34, 4, 8, 3, 5, 16, 18, 15, 3, 1, 2, 3, 4, 7, 13, 9, 4, 7, 1, 2, 23, 15, 11, 3, 5, 2, 3, 1, 8, 6, 13, 6, 16, 18, 5, 15, 3, 1, 18, 2, 3, 6, 4, 1, 2, 3, 4, 4, 1, 2, 1, 27, 8, 11, 8, 3, 4, 1, 14, 18, 2, 20, 1, 15, 11, 3, 4, 1, 1, 19, 9, 8, 4, 32, 7, 13, 3, 6, 6, 5, 16, 3, 21, 11, 3, 1, 2, 3, 1, 1, 2, 4, 1, 7, 3, 1, 2, 4, 5, 13, 4, 4, 1, 14, 8, 9, 2, 5, 2, 4, 4, 5, 15, 5, 7, 19, 8, 6, 6, 8, 4, 7, 3, 1, 14, 15, 3, 1, 2, 5, 2, 3, 1, 13, 1, 2, 3, 1, 8, 2, 9, 7, 3, 5, 2, 4, 7, 4, 13, 4, 5, 2, 1, 12, 9, 4, 2, 3, 1, 1, 2, 8, 1, 14, 8, 1, 13, 17, 4, 5, 8, 13, 3, 5, 13, 4, 4, 5, 33, 5, 32, 2, 10, 16, 9, 18, 16, 2, 15, 11, 3, 28, 1, 2, 3, 1, 2, 3, 4, 1, 8, 3, 6, 6, 2, 6, 31, 13, 31, 33, 6, 32, 3, 4, 4, 1, 22, 14, 3, 4, 1, 2, 1, 1, 14, 18, 5, 33, 3, 9, 4, 9, 36, 17, 32, 2, 5, 2, 4, 5, 15, 11, 15, 3, 4, 17, 8, 6, 14, 18, 2, 5, 2, 4, 5, 7, 19, 8, 1, 14, 8, 1, 14, 8, 1, 11, 8, 11,

In [7]:
data = pad_sequences(sequences, maxlen=maxlen, padding='post', truncating='post')
print(data[0])
print(np.shape(data))

[23 10 11  3  1  1  7 14  3  4  1 22 14 15 11  3  4 28  1  7  3 28  1 13
  3 28  1  1 13 22 16 10 16 34  4  8  3  5 16 18 15  3  1  2  3  4  7 13
  9  4  7  1  2 23 15 11  3  5  2  3  1  8  6 13  6 16 18  5 15  3  1 18
  2  3  6  4  1  2  3  4  4  1  2  1 27  8 11  8  3  4  1 14 18  2 20  1
 15 11  3  4  1  1 19  9  8  4 32  7 13  3  6  6  5 16  3 21 11  3  1  2
  3  1  1  2  4  1  7  3  1  2  4  5 13  4  4  1 14  8  9  2  5  2  4  4
  5 15  5  7 19  8  6  6  8  4  7  3  1 14 15  3  1  2  5  2  3  1 13  1
  2  3  1  8  2  9  7  3  5  2  4  7  4 13  4  5  2  1 12  9  4  2  3  1
  1  2  8  1 14  8  1 13 17  4  5  8 13  3  5 13  4  4  5 33  5 32  2 10
 16  9 18 16  2 15 11  3 28  1  2  3  1  2  3  4  1  8  3  6  6  2  6 31
 13 31 33  6 32  3  4  4  1 22 14  3  4  1  2  1  1 14 18  5 33  3  9  4
  9 36 17 32  2  5  2  4  5 15 11 15  3  4 17  8  6 14 18  2  5  2  4  5
  7 19  8  1 14  8  1 14  8  1 11  8 11  8 13 20 34  4  1 14  2 20  4  1
  8  6 13  6 16 18  2  3  8  1  1 15 11 18  7 19  3

In [8]:
print(tags[0])
tags = np.array(tags)
print(tags.shape)

2
(567,)


In [9]:
data = to_categorical(data)
print(data.shape)
print(data[0])

(567, 1000, 45)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


In [10]:
np.random.seed(0)
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
tags = tags[indices]
print(data.shape)
print(tags.shape)

(567, 1000, 45)
(567,)


In [11]:
model = Sequential()
model.add(Conv1D(128, 5, activation='relu',kernel_regularizer=l1_l2(l1=0.01, l2=0.0)))
model.add(MaxPooling1D(5))
model.add(Conv1D(64, 5, activation='relu',kernel_regularizer=l1_l2(l1=0.007, l2=0.0)))
model.add(GlobalMaxPooling1D())
model.add(Dense(64, activation='relu'))
model.add(Dense(1))
# model.summary()
def soft_acc(y_true, y_pred):
    from tensorflow.python.keras import backend as K
    return K.mean(K.equal(K.round(y_true), K.round(y_pred)))

In [ ]:
es = EarlyStopping(monitor='acc',patience=20, mode='auto')
model.compile(optimizer=RMSprop(),
loss='mse',
metrics=[soft_acc])
history = model.fit(data, tags,
epochs=1000,  
batch_size=50,
validation_split=0.2,
callbacks=[])
# model.save_weights('pre_trained_glove_model.h5')
model.summary()

Train on 453 samples, validate on 114 samples
Epoch 1/1000
453/453 [==============================] - 2s 4ms/step - loss: 20.4785 - soft_acc: 0.3289 - val_loss: 16.9729 - val_soft_acc: 0.3246
Epoch 2/1000
453/453 [==============================] - 0s 616us/step - loss: 14.8087 - soft_acc: 0.3510 - val_loss: 12.6459 - val_soft_acc: 0.3246
Epoch 3/1000
453/453 [==============================] - 0s 610us/step - loss: 10.9078 - soft_acc: 0.3466 - val_loss: 8.9385 - val_soft_acc: 0.3070
Epoch 4/1000
453/453 [==============================] - 0s 610us/step - loss: 7.7048 - soft_acc: 0.3400 - val_loss: 6.2511 - val_soft_acc: 0.3070
Epoch 5/1000
453/453 [==============================] - 0s 611us/step - loss: 5.2111 - soft_acc: 0.3400 - val_loss: 4.1260 - val_soft_acc: 0.3070
Epoch 6/1000
453/453 [==============================] - 0s 607us/step - loss: 3.3232 - soft_acc: 0.3400 - val_loss: 2.5406 - val_soft_acc: 0.3070
Epoch 7/1000
453/453 [==============================] - 0s 619us/step - los

453/453 [==============================] - 0s 601us/step - loss: 0.9626 - soft_acc: 0.3400 - val_loss: 0.9994 - val_soft_acc: 0.3070
Epoch 57/1000
453/453 [==============================] - 0s 604us/step - loss: 0.9587 - soft_acc: 0.3400 - val_loss: 0.9812 - val_soft_acc: 0.3070
Epoch 58/1000
453/453 [==============================] - 0s 606us/step - loss: 0.9591 - soft_acc: 0.3400 - val_loss: 0.9966 - val_soft_acc: 0.3070
Epoch 59/1000
453/453 [==============================] - 0s 606us/step - loss: 0.9553 - soft_acc: 0.3400 - val_loss: 0.9918 - val_soft_acc: 0.3070
Epoch 60/1000
453/453 [==============================] - 0s 610us/step - loss: 0.9547 - soft_acc: 0.3400 - val_loss: 0.9810 - val_soft_acc: 0.3070
Epoch 61/1000
453/453 [==============================] - 0s 607us/step - loss: 0.9541 - soft_acc: 0.3400 - val_loss: 1.0204 - val_soft_acc: 0.3070
Epoch 62/1000
453/453 [==============================] - 0s 609us/step - loss: 0.9620 - soft_acc: 0.3400 - val_loss: 0.9864 - val_so

Epoch 112/1000
453/453 [==============================] - 0s 621us/step - loss: 0.9496 - soft_acc: 0.3400 - val_loss: 0.9856 - val_soft_acc: 0.3070
Epoch 113/1000
453/453 [==============================] - 0s 646us/step - loss: 0.9571 - soft_acc: 0.3400 - val_loss: 0.9780 - val_soft_acc: 0.3070
Epoch 114/1000
453/453 [==============================] - 0s 588us/step - loss: 0.9500 - soft_acc: 0.3400 - val_loss: 0.9914 - val_soft_acc: 0.3070
Epoch 115/1000
453/453 [==============================] - 0s 571us/step - loss: 0.9505 - soft_acc: 0.3400 - val_loss: 0.9982 - val_soft_acc: 0.3070
Epoch 116/1000
453/453 [==============================] - 0s 573us/step - loss: 0.9498 - soft_acc: 0.3400 - val_loss: 0.9893 - val_soft_acc: 0.3070
Epoch 117/1000
453/453 [==============================] - 0s 579us/step - loss: 0.9509 - soft_acc: 0.3400 - val_loss: 0.9775 - val_soft_acc: 0.3070
Epoch 118/1000
453/453 [==============================] - 0s 568us/step - loss: 0.9557 - soft_acc: 0.3400 - val_

453/453 [==============================] - 0s 572us/step - loss: 0.9469 - soft_acc: 0.3400 - val_loss: 0.9801 - val_soft_acc: 0.3070
Epoch 168/1000
453/453 [==============================] - 0s 586us/step - loss: 0.9443 - soft_acc: 0.3400 - val_loss: 1.0022 - val_soft_acc: 0.3070
Epoch 169/1000
453/453 [==============================] - 0s 626us/step - loss: 0.9482 - soft_acc: 0.3400 - val_loss: 0.9735 - val_soft_acc: 0.3070
Epoch 170/1000
453/453 [==============================] - 0s 645us/step - loss: 0.9435 - soft_acc: 0.3400 - val_loss: 0.9809 - val_soft_acc: 0.3070
Epoch 171/1000
453/453 [==============================] - 0s 645us/step - loss: 0.9497 - soft_acc: 0.3400 - val_loss: 0.9747 - val_soft_acc: 0.3070
Epoch 172/1000
453/453 [==============================] - 0s 602us/step - loss: 0.9450 - soft_acc: 0.3400 - val_loss: 0.9848 - val_soft_acc: 0.3070
Epoch 173/1000
453/453 [==============================] - 0s 626us/step - loss: 0.9433 - soft_acc: 0.3400 - val_loss: 0.9745 - 

453/453 [==============================] - 0s 606us/step - loss: 0.9301 - soft_acc: 0.3400 - val_loss: 0.9842 - val_soft_acc: 0.3070
Epoch 223/1000
453/453 [==============================] - 0s 610us/step - loss: 0.9313 - soft_acc: 0.3400 - val_loss: 0.9686 - val_soft_acc: 0.3070
Epoch 224/1000
453/453 [==============================] - 0s 602us/step - loss: 0.9274 - soft_acc: 0.3400 - val_loss: 0.9747 - val_soft_acc: 0.3070
Epoch 225/1000
453/453 [==============================] - 0s 615us/step - loss: 0.9399 - soft_acc: 0.3400 - val_loss: 0.9659 - val_soft_acc: 0.3070
Epoch 226/1000
453/453 [==============================] - 0s 610us/step - loss: 0.9313 - soft_acc: 0.3400 - val_loss: 0.9645 - val_soft_acc: 0.3070
Epoch 227/1000
453/453 [==============================] - 0s 612us/step - loss: 0.9268 - soft_acc: 0.3400 - val_loss: 0.9643 - val_soft_acc: 0.3070
Epoch 228/1000
453/453 [==============================] - 0s 636us/step - loss: 0.9276 - soft_acc: 0.3400 - val_loss: 0.9765 - 

...

453/453 [==============================] - 0s 608us/step - loss: 0.5796 - soft_acc: 0.7285 - val_loss: 0.9774 - val_soft_acc: 0.5439
Epoch 608/1000
453/453 [==============================] - 0s 625us/step - loss: 0.6051 - soft_acc: 0.7020 - val_loss: 0.9882 - val_soft_acc: 0.5351
Epoch 609/1000
453/453 [==============================] - 0s 612us/step - loss: 0.5992 - soft_acc: 0.7020 - val_loss: 0.9729 - val_soft_acc: 0.5175
Epoch 610/1000
453/453 [==============================] - 0s 609us/step - loss: 0.6002 - soft_acc: 0.7196 - val_loss: 1.0526 - val_soft_acc: 0.4474
Epoch 611/1000
453/453 [==============================] - 0s 604us/step - loss: 0.5856 - soft_acc: 0.7307 - val_loss: 1.0346 - val_soft_acc: 0.5263
Epoch 612/1000
453/453 [==============================] - 0s 610us/step - loss: 0.6004 - soft_acc: 0.7020 - val_loss: 1.1117 - val_soft_acc: 0.3947
Epoch 613/1000
453/453 [==============================] - 0s 606us/step - loss: 0.5898 - soft_acc: 0.7108 - val_loss: 1.0159 - 

453/453 [==============================] - 0s 854us/step - loss: 0.5698 - soft_acc: 0.7704 - val_loss: 0.9970 - val_soft_acc: 0.5000
Epoch 663/1000
453/453 [==============================] - 0s 877us/step - loss: 0.5791 - soft_acc: 0.7461 - val_loss: 0.9758 - val_soft_acc: 0.5351
Epoch 664/1000
453/453 [==============================] - 0s 899us/step - loss: 0.5737 - soft_acc: 0.7351 - val_loss: 1.0086 - val_soft_acc: 0.4649
Epoch 665/1000
453/453 [==============================] - 0s 867us/step - loss: 0.5780 - soft_acc: 0.7461 - val_loss: 1.2232 - val_soft_acc: 0.4035
Epoch 666/1000
453/453 [==============================] - 0s 924us/step - loss: 0.5748 - soft_acc: 0.7682 - val_loss: 1.0063 - val_soft_acc: 0.5088
Epoch 667/1000
453/453 [==============================] - 0s 905us/step - loss: 0.5690 - soft_acc: 0.7682 - val_loss: 1.0461 - val_soft_acc: 0.5000
Epoch 668/1000
453/453 [==============================] - 0s 877us/step - loss: 0.5747 - soft_acc: 0.7550 - val_loss: 1.2000 - 

453/453 [==============================] - 0s 580us/step - loss: 0.5726 - soft_acc: 0.7528 - val_loss: 1.0991 - val_soft_acc: 0.4561
Epoch 718/1000
453/453 [==============================] - 0s 571us/step - loss: 0.5572 - soft_acc: 0.7638 - val_loss: 1.0549 - val_soft_acc: 0.4825
Epoch 719/1000
453/453 [==============================] - 0s 582us/step - loss: 0.5699 - soft_acc: 0.7528 - val_loss: 1.0597 - val_soft_acc: 0.4737
Epoch 720/1000
453/453 [==============================] - 0s 571us/step - loss: 0.5616 - soft_acc: 0.7704 - val_loss: 1.2934 - val_soft_acc: 0.4035
Epoch 721/1000
453/453 [==============================] - 0s 577us/step - loss: 0.5789 - soft_acc: 0.7461 - val_loss: 1.0169 - val_soft_acc: 0.4825
Epoch 722/1000
453/453 [==============================] - 0s 564us/step - loss: 0.5630 - soft_acc: 0.7748 - val_loss: 1.0584 - val_soft_acc: 0.4912
Epoch 723/1000
453/453 [==============================] - 0s 583us/step - loss: 0.5691 - soft_acc: 0.7770 - val_loss: 1.0215 - 

453/453 [==============================] - 0s 560us/step - loss: 0.5520 - soft_acc: 0.7815 - val_loss: 1.0113 - val_soft_acc: 0.4912
Epoch 773/1000
453/453 [==============================] - 0s 566us/step - loss: 0.5692 - soft_acc: 0.7439 - val_loss: 1.0340 - val_soft_acc: 0.4649
Epoch 774/1000
453/453 [==============================] - 0s 572us/step - loss: 0.5571 - soft_acc: 0.7792 - val_loss: 1.1483 - val_soft_acc: 0.4386
Epoch 775/1000
453/453 [==============================] - 0s 603us/step - loss: 0.5675 - soft_acc: 0.7616 - val_loss: 1.0408 - val_soft_acc: 0.4825
Epoch 776/1000
453/453 [==============================] - 0s 602us/step - loss: 0.5620 - soft_acc: 0.7770 - val_loss: 1.0917 - val_soft_acc: 0.4649
Epoch 777/1000
453/453 [==============================] - 0s 576us/step - loss: 0.5626 - soft_acc: 0.7616 - val_loss: 1.0093 - val_soft_acc: 0.4825
Epoch 778/1000
453/453 [==============================] - 0s 592us/step - loss: 0.5554 - soft_acc: 0.7660 - val_loss: 1.1226 - 

453/453 [==============================] - 0s 589us/step - loss: 0.5512 - soft_acc: 0.7682 - val_loss: 1.0634 - val_soft_acc: 0.4825
Epoch 828/1000
453/453 [==============================] - 0s 576us/step - loss: 0.5452 - soft_acc: 0.7748 - val_loss: 1.0998 - val_soft_acc: 0.4912
Epoch 829/1000
453/453 [==============================] - 0s 589us/step - loss: 0.5522 - soft_acc: 0.7748 - val_loss: 1.2286 - val_soft_acc: 0.4211
Epoch 830/1000
453/453 [==============================] - 0s 578us/step - loss: 0.5764 - soft_acc: 0.7528 - val_loss: 1.0438 - val_soft_acc: 0.5000
Epoch 831/1000
453/453 [==============================] - 0s 580us/step - loss: 0.5496 - soft_acc: 0.7881 - val_loss: 1.0550 - val_soft_acc: 0.4737
Epoch 832/1000
453/453 [==============================] - 0s 586us/step - loss: 0.5429 - soft_acc: 0.7837 - val_loss: 0.9968 - val_soft_acc: 0.5175
Epoch 833/1000
453/453 [==============================] - 0s 582us/step - loss: 0.5513 - soft_acc: 0.7704 - val_loss: 1.1378 - 

In [ ]:
import matplotlib.pyplot as plt

soft_acc = history.history['soft_acc']
soft_val_acc = history.history['val_soft_acc']

acc = history.history['acc']
val_acc = history.history['val_acc']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)
plt.figure(figsize=(30,5))
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, soft_acc, 'ro', label='Soft Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.plot(epochs, soft_val_acc, 'r', label='Soft Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure(figsize=(30,5))
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()




In [ ]:
result = model.evaluate(data, tags)
print(result[1])